In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [226]:
player_data = pd.read_csv('..\\Consolidated Data\\Player_Attribute_Data.csv')
team_rankings = pd.read_csv('..\\Consolidated Data\\Team Rankings_corrected.csv')
spend_df = pd.read_csv('..\\Consolidated Data\\Average_Transfer_Budget.csv')

In [227]:
team_rankings=team_rankings.drop(['Unnamed: 0'],axis=1)
player_df = player_data[['sofifa_id','club', 'overall','position', 'section', 'role','value_eur','age','short_name']]
combo_df = pd.merge(player_df,team_rankings,how='inner',left_on='club',right_on='Team')
combo_df=combo_df.drop(['sofifa_id','club','Team'],axis=1)
role_df = combo_df.groupby(['role','Tier']).mean()
pos_df = combo_df.groupby(['position','Tier']).mean()
role_df.reset_index(inplace = True)
pos_df.reset_index(inplace = True)

In [228]:
def club_level(club):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    if(club_tier!=1):
        club_df = player_data[player_data['club']==club][['overall','section','role']]
        club_df = club_df.groupby(['role']).mean()
        club_df = club_df.reset_index()
        upper_tier_df = role_df[role_df['Tier']==(club_tier-1)]
        upper_tier_df = upper_tier_df.reset_index()
        upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
        upper_tier_df = upper_tier_df[['role','overall']]
        combo_df = pd.merge(upper_tier_df,club_df,how='inner',on='role')
        combo_df.columns = ['Role','Upper_Tier_Rating','Club_Rating']
        combo_df['Rating_Gap'] = combo_df['Upper_Tier_Rating'] - combo_df['Club_Rating']
    else:
        club_df = player_data[player_data['club']==club][['overall','section','role']]
        club_df = club_df.groupby(['role']).mean()
        club_df = club_df.reset_index()
        mean_tier_df = role_df[role_df['Tier']==(club_tier)]
        mean_tier_df = mean_tier_df.reset_index()
        mean_tier_df = mean_tier_df.drop(['index','Tier'],axis=1)
        mean_tier_df = mean_tier_df[['role','overall']]
        combo_df = pd.merge(mean_tier_df,club_df,how='inner',on='role')
        combo_df.columns = ['Role','Mean_Tier_Rating','Club_Rating']
        combo_df['Rating_Gap'] = combo_df['Mean_Tier_Rating'] - combo_df['Club_Rating']
    role_focus = combo_df[combo_df['Rating_Gap']==combo_df['Rating_Gap'].values.max()]['Role'].values[0]
    return role_focus

In [229]:
def club_gap(club):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    if (club_tier!=1):
        club_df = player_data[player_data['club']==club][['overall','section','position']]
        club_df = club_df.groupby(['position']).mean()
        club_df = club_df.reset_index()
        upper_tier_df = pos_df[pos_df['Tier']==(club_tier-1)]
        upper_tier_df = upper_tier_df.reset_index()
        upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
        upper_tier_df = upper_tier_df[['role','overall']]
        combined_df = pd.merge(upper_tier_df,club_df,how='inner',on='position')
        combined_df.head(1)
        combined_df.columns = ['Position','Upper_Tier_Rating','Club_Rating']
        combined_df['Rating_Gap'] = combined_df['Upper_Tier_Rating'] - combined_df['Club_Rating']
    else:
        club_df = player_data[player_data['club']==club][['overall','section','position']]
        club_df = club_df.groupby(['position']).mean()
        club_df = club_df.reset_index()
        mean_tier_df = pos_df[pos_df['Tier']==(club_tier)]
        mean_tier_df = upper_tier_df.reset_index()
        mean_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
        mean_tier_df = mean_tier_df[['role','overall']]
        combined_df = pd.merge(mean_tier_df,club_df,how='inner',on='position')
        combined_df.head(1)
        combined_df.columns = ['Position','Mean_Tier_Rating','Club_Rating']
        combined_df['Rating_Gap'] = combined_df['Mean_Tier_Rating'] - combined_df['Club_Rating']
    pos_focus = combined_df[combined_df['Rating_Gap']==combined_df['Rating_Gap'].values.max()]['Position'].values[0]
    return pos_focus

In [230]:
def club_gap_2(club,pr):
    club_tier = (team_rankings[team_rankings['Team'] == club]['Tier']).values[0]
    if(club_tier!=1):
        club_df = player_data[player_data['club']==club][['overall','section','position']]
        club_df = club_df.groupby(['position']).mean()
        club_df = club_df.reset_index()
        upper_tier_df = pos_df[pos_df['Tier']==(club_tier-1)]
        upper_tier_df = upper_tier_df.reset_index()
        upper_tier_df = upper_tier_df.drop(['index','Tier'],axis=1)
        upper_tier_df = upper_tier_df[['position','overall']]
        combined_df = pd.merge(upper_tier_df,club_df,how='inner',on='position')
        combined_df.head(1)
        combined_df.columns = ['Position','Upper_Tier_Rating','Club_Rating']
        combined_df['Rating_Gap'] = combined_df['Upper_Tier_Rating'] - combined_df['Club_Rating']
    else:
        club_df = player_data[player_data['club']==club][['overall','section','position']]
        club_df = club_df.groupby(['position']).mean()
        club_df = club_df.reset_index()
        mean_tier_df = pos_df[pos_df['Tier']==(club_tier)]
        mean_tier_df = mean_tier_df.reset_index()
        mean_tier_df = mean_tier_df.drop(['index','Tier'],axis=1)
        mean_tier_df = mean_tier_df[['position','overall']]
        combined_df = pd.merge(mean_tier_df,club_df,how='inner',on='position')
        combined_df.columns = ['Position','Mean_Tier_Rating','Club_Rating']
        combined_df['Rating_Gap'] = combined_df['Mean_Tier_Rating'] - combined_df['Club_Rating']        
    pos_focus = combined_df[combined_df['Rating_Gap'] == np.sort(combined_df['Rating_Gap'])[-1*pr]]['Position'].values[0]
    return pos_focus

In [231]:
def rec_player_by_role(club,pct):
    spend_limit = (spend_df[spend_df['Team']==club]['Total_Arrivals'].values[0]) * pct
    player_df['value_in_mil'] = player_df['value_eur']/1000000
    clean_df = player_df[player_df['club'] != club]
    limit_df=clean_df[clean_df['value_in_mil']<spend_limit]
    age_df = limit_df[['club','age']].groupby('club').mean()
    age_df['age'] = round(age_df['age'],2)
    age_df = age_df.reset_index()
    target_age = player_df[player_df['club'] == club]
    target_age=target_age['age'].values[0]
    role_to_fill=club_level(club)
    role_df = limit_df[clean_df['role']==role_to_fill]
    age_range_df = role_df[(role_df['age']>=(target_age-1.5)) & (role_df['age']<=(target_age+1.5))]
#     print(age_range_df.columns)
    return(age_range_df.head(1)['short_name'].values[0])

In [232]:
def rec_player_by_pos(club,pct,pr):
    spend_limit = (spend_df[spend_df['Team']==club]['Total_Arrivals'].values[0]) * pct
    player_df['value_in_mil'] = player_df['value_eur']/1000000
    clean_df = player_df[player_df['club'] != club]
    limit_df=clean_df[clean_df['value_in_mil']<spend_limit]
    age_df = limit_df[['club','age']].groupby('club').mean()
    age_df['age'] = round(age_df['age'],2)
    age_df = age_df.reset_index()
    target_age = player_df[player_df['club'] == club]
    target_age=target_age['age'].values[0]
    pos_to_fill = club_gap_2(club,pr)
    pos_df = limit_df[player_df['position']==pos_to_fill]
    age_range_df = pos_df[(pos_df['age']>=(target_age-1.5)) & (pos_df['age']<=(target_age+1.5))]
#     print(age_range_df.columns())
    return(age_range_df.head(1)['short_name'].values[0])

In [233]:
def recommend_players(club,count):
    player_array = []
    if(count == 1):
        player = rec_player_by_role(club,1)
        player_array.append(player)
        print(player_array)
    elif(count == 2):
        player = rec_player_by_role(club,.6)
        player_array.append(player)
        player = rec_player_by_pos(club,.4,1)
        player_array.append(player)
        print(player_array)
    elif(count == 3):
        player = rec_player_by_role(club,.45)
        player_array.append(player)
        player = rec_player_by_pos(club,.30,1)
        player_array.append(player)
        player = rec_player_by_role(club,.25)
        player_array.append(player)
        print(player_array)
    elif(count == 4):
        player = rec_player_by_role(club,.40)
        player_array.append(player)
        player = rec_player_by_pos(club,.30,1)
        player_array.append(player)
        player = rec_player_by_role(club,.20)
        player_array.append(player)
        player = rec_player_by_pos(club,.10,2)
        player_array.append(player)
        print(player_array)
    elif(count == 5):
        player = rec_player_by_role(club,.35)
        player_array.append(player)
        player = rec_player_by_pos(club,.28,1)
        player_array.append(player)
        player = rec_player_by_role(club,.21)
        player_array.append(player)
        player = rec_player_by_pos(club,.15,2)
        player_array.append(player)
        player = rec_player_by_pos(club,.10,3)
        player_array.append(player)
        print(player_array)

In [251]:
recommend_players('Juventus',1)

['David Silva']


c:\users\akhil_s_bhat\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [247]:
team_rankings.head(20)

,Team,Tier
0,Chelsea,1
1,FC Bayern München,1
2,FC Barcelona,1
3,Real Madrid,1
4,Manchester United,2
5,Tottenham Hotspur,2
6,Manchester City,2
7,Atlético Madrid,2
8,Juventus,2
9,Liverpool,2
